Checking the parameters before

In [ ]:
"""
CARMEN Trace Analysis Script – Onset, Return, and Oscillation Detection
-----------------------------------------------------------------------

This script performs post-processing of calcium multiplexing data exported from the CARMEN Fiji macro and 
post processed with CARMEN_combine_and_curvefitting_xlsmtemplate.ipynb .

For each input Excel file (.xlsm), the script:
- Loads time series and fluorescence intensity data.
- Applies Savitzky-Golay smoothing to reduce noise in signal traces.
- Calculates the first derivative of each smoothed trace to identify activation onset times.
- Defines a baseline region and calculates the time taken for the signal to return to baseline.
- Detects oscillations (peaks and troughs) between onset and return.
- Plots raw, smoothed, and derivative traces for visual inspection, highlighting key features.

Key Features:
- Derivative-based onset detection using `scipy.signal.find_peaks`, adjustable for signal polarity.
- Baseline return detection using threshold-based criteria with configurable tolerance and duration.
- Detection of oscillations between onset and return using prominence-based peak/trough filtering.
- High-resolution visualisation of signal features across time.

Assumptions:
- Input files are `.xlsm` Excel workbooks with a sheet named 'Time Analysis'.
- Time is stored in the column named 'Protocol Time [sec]'.
- Signal data are found in columns C, D, and E (representing GFP, RFP, and NIR).
- Signals are ordered consistently across files, and rows with NaN values are excluded automatically.

Usage:
- Set `folder_path` to the directory containing your input `.xlsm` files.
- Adjust analysis parameters at the top of the script to suit your experimental design.
- Run the script in a Python environment with the required dependencies.

Output:
- Printed console summaries of detected onset, return, and duration per signal.
- Matplotlib plots showing raw, smoothed, and derivative signals with annotated onsets and returns.

"""
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from scipy.signal import find_peaks, savgol_filter

# === USER PARAMETERS === #

# Smoothing constants for Savitzky-Golay filter
window_length = 15 # must be an odd number
polyorder = 2 # polynomial order for smoothing
window_length_strong = 15
polyorder_strong = 2

# Return to baseline detection sensitivity
threshold_percentage = 0.3 # how close to baseline the signal must return
min_consecutive_points = 15 # number of points within the threshold to confirm return

# Baseline definition window (in seconds) 
baseline_start_time = 200
baseline_end_time = 320

# Define peak direction per signal: +1 = looking for peaks, -1 = looking for troughs
gfp_peak_or_trough = +1  # GFP signal is positive (looking for peaks)
rfp_peak_or_trough = -1  # RFP signal is negative (looking for troughs)
nir_peak_or_trough = +1  # NIR signal is positive (looking for peaks)

# Peak detection settings for onset (derivative-based)
gfp_params = {'height': 0.03, 'distance': 5, 'prominence': 0.2, 'width': 5}  # GFP derivative onset peak detection
rfp_params = {'height': 0.02, 'distance': 5, 'prominence': 0.2, 'width': 3}  # RFP derivative onset peak detection
nir_params = {'height': 0.02, 'distance': 5, 'prominence': 0.4, 'width': 5}  # NIR derivative onset peak detection

# Peak detection settings for oscillations (raw signals)
gfp_peak_prominence = 0.3
gfp_peak_distance = 5
rfp_peak_prominence = 0.3
rfp_peak_distance = 5
nir_peak_prominence = 0.3
nir_peak_distance = 5

# X-axis zoom range for plotting to focus on the specified range in seconds
zoomed_min = 320 
zoomed_max = 450

# === INPUT DATA FOLDER === #
folder_path = r"your\file\path" 
file_list = [file for file in os.listdir(folder_path) if file.endswith('.xlsm')]

if not file_list:
    print(f"No '.xlsm' files found in the folder: {folder_path}")
else:
    print(f"Found {len(file_list)} files: {file_list}")

# === PROCESS EACH FILE === #
for file_name in file_list:
    print(f"Processing file: {file_name}") 
    file_path = os.path.join(folder_path, file_name)
    # Load data from Excel
    df = pd.read_excel(file_path, sheet_name='Time Analysis')
    
    # Load time in seconds from column B
    time_seconds = df['Protocol Time [sec]'].to_numpy()
    
    data_sets = []  # To store data sets for the current file
    results = []  # To store the results for the current file

    # Load intensity data from columns C-D-E, F-G-H, etc.
    intensity_set = df.iloc[:, 2:5].to_numpy()  # Extract columns C-D-E
    
    # Filter intensity data and time; Handle NaN values
    mask = ~np.isnan(intensity_set).any(axis=1)  # Mask to filter out rows with NaN values
    filtered_time = time_seconds[mask]           # Filter time values
    filtered_intensity = intensity_set[mask]     # Filter intensity values
        
    if len(filtered_intensity) > 0:  # Check if the intensity data is not empty
        data_sets.append((filtered_time, filtered_intensity))

    # Analyze each data set for the current file
    for idx, (time, intensity) in enumerate(data_sets):
        print(f"  Processing dataset {idx + 1} in file: {file_name}")
        gfp, rfp, nir = intensity.T  # Split intensity data into gfp, rfp, and nir
        
        # Step 1: Smooth the data using with Savitzky-Golay filter
        gfp_smooth = savgol_filter(gfp, window_length, polyorder)
        rfp_smooth = savgol_filter(rfp, window_length, polyorder)
        nir_smooth = savgol_filter(nir, window_length, polyorder)
        
        # Stronger smoothing (for return-to-baseline analysis)          
        gfp_smooth_strong = savgol_filter(gfp, window_length=window_length_strong, polyorder=polyorder_strong)
        rfp_smooth_strong = savgol_filter(rfp, window_length=window_length_strong, polyorder=polyorder_strong)
        nir_smooth_strong = savgol_filter(nir, window_length=window_length_strong, polyorder=polyorder_strong)

        # Step 2: Compute derivatives to detect onsets
        gfp_derivative = np.gradient(gfp_smooth, time)
        rfp_derivative = np.gradient(rfp_smooth, time)
        nir_derivative = np.gradient(nir_smooth, time)
        
        # Step 3: Detect onset times
        # Multiply the derivatives by the respective direction
        gfp_adjusted_derivative = gfp_peak_or_trough * gfp_derivative
        rfp_adjusted_derivative = rfp_peak_or_trough * rfp_derivative
        nir_adjusted_derivative = nir_peak_or_trough * nir_derivative
        #Detect peaks in the derivative
        gfp_peaks_derivative, _ = find_peaks(gfp_adjusted_derivative, **gfp_params)
        rfp_peaks_derivative, _ = find_peaks(rfp_adjusted_derivative, **rfp_params)
        nir_peaks_derivative, _ = find_peaks(nir_adjusted_derivative, **nir_params)
        # Assign onset times if peaks or troughs were detected        
        gfp_onset_time = time[gfp_peaks_derivative[0]] if len(gfp_peaks_derivative) > 0 else None
        rfp_onset_time = time[rfp_peaks_derivative[0]] if len(rfp_peaks_derivative) > 0 else None
        nir_onset_time = time[nir_peaks_derivative[0]] if len(nir_peaks_derivative) > 0 else None
        
        # Step 4: Compute return time and global durations
                
        # Function to define the baseline from a specific time range
        def define_baseline(signal, time, baseline_start_time, baseline_end_time):
            # Find the indices corresponding to the specified time range
            baseline_indices = np.where((time >= baseline_start_time) & (time <= baseline_end_time))[0]

            # Ensure that valid indices were found
            if len(baseline_indices) == 0:
                raise ValueError("No valid indices found for baseline calculation.")
            
            # Ensure the start time is before the end time
            if baseline_start_time >= baseline_end_time:
                raise ValueError("Baseline start time should be less than baseline end time.")
            
            # Check if indices are within the bounds of the signal
            if max(baseline_indices) >= len(signal):
                raise IndexError("Baseline indices are out of bounds of the signal length.")
            
            # Calculate and return the mean of the specified baseline range
            return np.mean(signal[baseline_indices])
        
        # Function to calculate signal duration from onset to baseline return
        def calculate_signal_duration(time, smoothed_signal, baseline, onset_time, signal_type, 
                                    is_low_baseline, min_consecutive_points=min_consecutive_points, 
                                    threshold_percentage=threshold_percentage):    
            # Handle the case where onset_time is None, e.g., no onset time could be identified automatically
            onset_time_candidate = onset_time
            if onset_time_candidate is None:
                print(f"{signal_type} onset_time is None. Using a fixed value of 60 seconds.")
                onset_time_candidate = 360  # Assign default onset_time

            # Find the onset index using a tolerance to match the onset time
            onset_index = (np.abs(time - onset_time_candidate)).argmin()

            # Start searching for the return to baseline at least 30 data points after the onset
            start_search_index = onset_index + 30

            # Define the lower and upper threshold based on the threshold percentage of the baseline
            lower_threshold = baseline * (1 - threshold_percentage)
            upper_threshold = baseline * (1 + threshold_percentage)

            # Initialize variables to track consecutive points within threshold
            consecutive_points = 0
            tracked_value_idx = None

            # Loop through the signal after the onset to find where the signal returns to baseline range
            for idx in range(start_search_index, len(smoothed_signal)):
                # Check if the signal is within the range of lower and upper thresholds
                if lower_threshold <= smoothed_signal[idx] <= upper_threshold:
                    consecutive_points += 1

                    # Track maximum or minimum value within the valid range based on `is_low_baseline`
                    if is_low_baseline:
                        # For GFP and NIR: Track minimum value
                        if tracked_value_idx is None or smoothed_signal[idx] < smoothed_signal[tracked_value_idx]:
                            tracked_value_idx = idx
                    else:
                        # For RFP: Track maximum value
                        if tracked_value_idx is None or smoothed_signal[idx] > smoothed_signal[tracked_value_idx]:
                            tracked_value_idx = idx

                else:
                    # Reset the consecutive points count if the signal leaves the threshold range
                    consecutive_points = 0
                    tracked_value_idx = None

                # If enough consecutive points are found, consider this a valid return to baseline
                if consecutive_points >= min_consecutive_points:
                    break  # Exit the loop once a valid return to baseline is found

            # Determine return time based on the tracked max or min value index
            if tracked_value_idx is not None:
                return_time = time[tracked_value_idx]
            else:
                return_time = np.nan  # If no valid return indices were found

            # Calculate the duration from onset to the return to baseline
            duration = return_time - onset_time_candidate

            # Return the onset time, return time, and calculated duration
            return onset_time_candidate, return_time, duration

        # Calculate baselines
        gfp_baseline = define_baseline(gfp, time, baseline_start_time, baseline_end_time)
        print(f"GFP baseline calculated for the range {baseline_start_time}s to {baseline_end_time}s: {gfp_baseline}")

        nir_baseline = define_baseline(nir, time, baseline_start_time, baseline_end_time)
        print(f"NIR baseline calculated for the range {baseline_start_time}s to {baseline_end_time}s: {nir_baseline}")

        rfp_baseline = define_baseline(rfp, time, baseline_start_time, baseline_end_time)
        print(f"RFP baseline calculated for the range {baseline_start_time}s to {baseline_end_time}s: {rfp_baseline}")

        # Determine if the baseline is low or high for each signal
        low_baseline_threshold = 20
        is_low_baseline_gfp = gfp_baseline < low_baseline_threshold
        is_low_baseline_nir = nir_baseline < low_baseline_threshold
        is_low_baseline_rfp = rfp_baseline < low_baseline_threshold

        # Calculate durations for GFP, RFP, and NIR
        # GFP: Determine if it's a low baseline and calculate return time accordingly
        gfp_onset_time, gfp_return, gfp_globalduration = calculate_signal_duration(
            time, gfp_smooth_strong, gfp_baseline, gfp_onset_time, signal_type='GFP', is_low_baseline=is_low_baseline_gfp)

        # NIR: Determine if it's a low baseline and calculate return time accordingly
        nir_onset_time, nir_return, nir_globalduration = calculate_signal_duration(
            time, nir_smooth_strong, nir_baseline, nir_onset_time, signal_type='NIR', is_low_baseline=is_low_baseline_nir)

        # RFP: Determine if it's a low baseline and calculate return time accordingly
        rfp_onset_time, rfp_return, rfp_globalduration = calculate_signal_duration(
            time, rfp_smooth_strong, rfp_baseline, rfp_onset_time, signal_type='RFP', is_low_baseline=is_low_baseline_rfp)

        # Print results
        print(f"GFP: Onset={gfp_onset_time}, Return={gfp_return}, Duration={gfp_globalduration}")
        print(f"NIR: Onset={nir_onset_time}, Return={nir_return}, Duration={nir_globalduration}")
        print(f"RFP: Onset={rfp_onset_time}, Return={rfp_return}, Duration={rfp_globalduration}")

        if gfp_onset_time and rfp_onset_time and nir_onset_time:
            delay_nir_rfp = rfp_onset_time - nir_onset_time
            delay_gfp_rfp = gfp_onset_time - rfp_onset_time
            delay_nir_gfp = gfp_onset_time - nir_onset_time   
        else:
            delay_nir_rfp = delay_gfp_rfp = delay_nir_gfp = None

        print(f"Delay NIR to RFP {delay_nir_rfp}, Delay GFP to RFP {delay_gfp_rfp}, Delay NIR to GFP {delay_nir_gfp}")
                
        
        
        # Step 5: Oscillation Detection and Count (Peaks and Troughs)
        # Function to detect peaks and troughs for a given signal

        def detect_oscillations(signal, prominence, distance, onset_time=None, return_time=None, lower_threshold=None, upper_threshold=None):
            """
            Detect peaks and troughs in the given signal using specified prominence, distance parameters,
            and optional lower and upper thresholds to reduce noise-induced detections near baseline.
            Additionally, only consider oscillations between onset and return times.

            Parameters:
            - signal: The input signal (numpy array)
            - prominence: The prominence of the peaks/troughs for detection
            - distance: Minimum distance between peaks/troughs
            - onset_time: The time to start looking for oscillations (e.g., after the onset)
            - return_time: The time to stop looking for oscillations (e.g., before the return time)
            - lower_threshold: Lower threshold value to ignore peaks/troughs near the baseline
            - upper_threshold: Upper threshold value to ignore peaks/troughs near the baseline

            Returns:
            - peaks: Indices of the peaks in the signal (between onset and return time)
            - troughs: Indices of the troughs in the signal (between onset and return time)
            """
            peaks = find_peaks(signal, prominence=prominence, distance=distance)[0]
            troughs = find_peaks(-signal, prominence=prominence, distance=distance)[0]

            # Convert onset and return times to indices
            if onset_time is not None:
                onset_index = (np.abs(time - onset_time)).argmin()
            else:
                onset_index = 0

            if return_time is not None:
                return_index = (np.abs(time - return_time)).argmin()
            else:
                return_index = len(signal)

            # Filter peaks and troughs to only include those between onset and return times
            peaks = [p for p in peaks if onset_index <= p <= return_index]
            troughs = [t for t in troughs if onset_index <= t <= return_index]

            # Filter out peaks and troughs that are within the lower and upper thresholds if provided
            #if lower_threshold is not None and upper_threshold is not None:
            #    peaks = [p for p in peaks if signal[p] < lower_threshold or signal[p] > upper_threshold]
            #    troughs = [t for t in troughs if signal[t] < lower_threshold or signal[t] > upper_threshold]

            if len(peaks) == 0 or len(troughs) == 0:
                print("Warning: No valid peaks or troughs detected for the signal.")

            return peaks, troughs

        # Detection and Duration Calculation for Each Signal (with onset and return time check)

        # Detect peaks and troughs for GFP signal (only between onset and return time)
        gfp_peaks, gfp_troughs = detect_oscillations(
            gfp_smooth_strong, prominence=gfp_peak_prominence, distance=gfp_peak_distance, onset_time=gfp_onset_time, return_time=gfp_return
        )
        gfp_oscillations = min(len(gfp_peaks), len(gfp_troughs))  # Each oscillation requires one peak and one trough

        # Detect peaks and troughs for RFP signal (only between onset and return time)
        rfp_peaks, rfp_troughs = detect_oscillations(
            rfp_smooth_strong, prominence=rfp_peak_prominence, distance=rfp_peak_distance, onset_time=rfp_onset_time, return_time=rfp_return
        )
        rfp_oscillations = min(len(rfp_peaks), len(rfp_troughs))

        # Detect peaks and troughs for NIR signal (only between onset and return time)
        nir_peaks, nir_troughs = detect_oscillations(
            nir_smooth_strong, prominence=nir_peak_prominence, distance=nir_peak_distance, onset_time=nir_onset_time, return_time=nir_return
        )
        nir_oscillations = min(len(nir_peaks), len(nir_troughs))

        # Function to calculate oscillation durations from the derivative of the signal
        def calculate_durations(derivative, time, signal_name, onset_time=None, return_time=None):
            """
            Calculate the durations of oscillations based on the changes in the derivative of the signal,
            considering only the oscillations between onset and return times.

            Parameters:
            - derivative: The derivative of the smoothed signal (numpy array)
            - time: Time vector corresponding to the signal (numpy array)
            - signal_name: Name of the signal being processed (str)
            - onset_time: The time to start calculating oscillations (e.g., after the onset)
            - return_time: The time to stop calculating oscillations (e.g., before the return time)

            Returns:
            - signal_durations: List of calculated durations for each oscillation
            """
            # Determine regions where the derivative changes sign (+1 for increase, -1 for decrease)
            change_regions = np.sign(derivative)

            # Detect changes in regions using np.diff; prepend zero to maintain the length
            transitions = np.diff(change_regions, prepend=0)

            # Identify indices where transitions occur, indicating peaks or troughs
            starts = np.where(transitions != 0)[0]

            # Convert onset and return times to indices
            if onset_time is not None:
                onset_index = (np.abs(time - onset_time)).argmin()
            else:
                onset_index = 0

            if return_time is not None:
                return_index = (np.abs(time - return_time)).argmin()
            else:
                return_index = len(derivative)

            # Filter out transitions that are outside the onset to return range
            starts = starts[(starts >= onset_index) & (starts <= return_index)]

            # Initialize list to store durations of detected oscillations
            signal_durations = []

            # Loop through the transitions to calculate each oscillation duration
            for i in range(len(starts) - 1):
                start_idx, end_idx = starts[i], starts[i + 1]

                # Validate that indices are within bounds of the time array
                if end_idx < len(time):
                    duration = int(time[end_idx] - time[start_idx])  # Convert duration to Python int
                    signal_durations.append(duration)
                else:
                    print(f"Warning: Index {end_idx} is out of bounds for time array of length {len(time)}.")

            return signal_durations

        # Compute durations for each signal (considering onset to return time)
        durations = {}

        # Calculate durations for each signal derivative (GFP, RFP, NIR)
        for derivative, name, onset_time, return_time in zip(
            [gfp_derivative, rfp_derivative, nir_derivative], 
            ["GFP", "RFP", "NIR"], 
            [gfp_onset_time, rfp_onset_time, nir_onset_time],
            [gfp_return, rfp_return, nir_return]
        ):
            durations[name] = calculate_durations(derivative, time, name, onset_time=onset_time, return_time=return_time)

    
        
        # Step 7: Plot curves, smoothed curves and derivative curves
        # First plot: Zoomed in on the time range from 120 to 180
        plt.figure(figsize=(12, 8))  # Use the same figure size as the first plot
        plt.plot(time, gfp, label="GFP (Original)", color='green', alpha=0.5)
        plt.plot(time, gfp_smooth, label="GFP (Smoothed)", color='green')
        plt.plot(time, gfp_derivative, label="GFP Derivative", color='lime', linestyle='--')        
        plt.plot(time, rfp, label="RFP (Original)", color='orange', alpha=0.5)
        plt.plot(time, rfp_smooth, label="RFP (Smoothed)", color='orange')
        plt.plot(time, rfp_derivative, label="RFP Derivative", color='gold', linestyle='--')
        plt.plot(time, nir, label="NIR (Original)", color='darkred', alpha=0.5)
        plt.plot(time, nir_smooth, label="NIR (Smoothed)", color='darkred')
        plt.plot(time, nir_derivative, label="NIR Derivative", color='red', linestyle='--')   
        # Mark detected onsets
        if gfp_onset_time: plt.axvline(gfp_onset_time, color='green', linestyle='--', label="GFP Onset")
        if rfp_onset_time: plt.axvline(rfp_onset_time, color='orange', linestyle='--', label="RFP Onset")
        if nir_onset_time: plt.axvline(nir_onset_time, color='darkred', linestyle='--', label="NIR Onset")        
        plt.xlabel("Time [sec]")
        plt.ylabel("Intensity")        
        plt.legend(loc='upper left', fontsize=6)        
        
        #Set the x-axis limits to focus on the specified range in seconds
        plt.xlim(zoomed_min, zoomed_max)
        
        # Set the grid for x-axis with major and minor ticks
        plt.grid(True, which='major', axis='x', color='lightgrey', linestyle='-', linewidth=0.5)  # Major grid lines
        plt.grid(True, which='minor', axis='x', color='lightgrey', linestyle='--', linewidth=0.3)  # Minor grid lines

        # Set major ticks (labels) every 10 seconds
        plt.gca().xaxis.set_major_locator(MultipleLocator(10))

        # Set minor ticks (grid lines) every 2 seconds
        plt.gca().xaxis.set_minor_locator(MultipleLocator(2))

        # Format minor ticks on the x-axis
        plt.tick_params(axis='x', which='minor', length=4, width=0.8, labelsize=8, labelbottom=False)  # Minor ticks, no labels
        plt.tick_params(axis='x', which='major', length=6, width=1.2, labelsize=10)  # Major ticks with labels

        # Set grid for y-axis
        plt.grid(True, which='both', axis='y', color='lightgrey', linestyle='-', linewidth=0.5)
        plt.show()
                
        #Second Plot: Oscillations        
        plt.figure(figsize=(12, 8))        
        plt.plot(time, gfp, label="GFP (Original)", color='green', alpha=0.5)
        plt.plot(time, gfp_smooth_strong, label="GFP (Smoothed)", color='green')
        plt.plot(time, gfp_derivative, label="GFP Derivative", color='lime', linestyle='--')        
        plt.plot(time, rfp, label="RFP (Original)", color='orange', alpha=0.5)
        plt.plot(time, rfp_smooth_strong, label="RFP (Smoothed)", color='orange')
        plt.plot(time, rfp_derivative, label="RFP Derivative", color='gold', linestyle='--')        
        plt.plot(time, nir, label="NIR (Original)", color='darkred', alpha=0.5)
        plt.plot(time, nir_smooth_strong, label="NIR (Smoothed)", color='darkred')
        plt.plot(time, nir_derivative, label="NIR Derivative", color='red', linestyle='--')
       
        #Mark detected peaks with scatter points
        for peak in gfp_peaks:
            plt.scatter(time[peak], gfp[peak], color='green', zorder=5, label="GFP Peak" if peak == gfp_peaks_derivative[0] else "")  # Only label the first peak
        for peak in rfp_peaks:
            plt.scatter(time[peak], rfp[peak], color='orange', zorder=5, label="RFP Peak" if peak == rfp_peaks_derivative[0] else "")  # Only label the first peak
        for peak in nir_peaks:
            plt.scatter(time[peak], nir[peak], color='darkred', zorder=5, label="NIR Peak" if peak == nir_peaks_derivative[0] else "")  # Only label the first peak
        #Mark detected troughs with scatter points
        for trough in gfp_troughs:
            plt.scatter(time[trough], gfp[trough], color='lime', zorder=5, label="GFP Trough" if trough == gfp_troughs[0] else "")  # Only label the first trough
        for trough in rfp_troughs:
            plt.scatter(time[trough], rfp[trough], color='gold', zorder=5, label="RFP Trough" if trough == rfp_troughs[0] else "")  # Only label the first trough
        for trough in nir_troughs:
            plt.scatter(time[trough], nir[trough], color='red', zorder=5, label="NIR Trough" if trough == nir_troughs[0] else "")  # Only label the first trough
        # Boolean flags to control labeling
        gfp_peak_label = True
        rfp_peak_label = True
        nir_peak_label = True
        gfp_trough_label = True
        rfp_trough_label = True
        nir_trough_label = True
        #label and legend
        plt.xlabel("Time [sec]", fontsize=10)
        plt.ylabel("Intensity", fontsize=10)
        plt.legend(loc='upper right', fontsize=6) 
        # Format the x-axis: label every 5 seconds starting from the first data point
        plt.xticks(np.arange(time[0], max(time), 30))  # Set x-ticks every 30 seconds starting from the first time point
        plt.grid(True, which='both', axis='x', color='lightgrey', linestyle='-', linewidth=0.2)
        plt.grid(True, which='both', axis='y', color='lightgrey', linestyle='-', linewidth=0.2)
        plt.yticks(fontsize=10)
        plt.show()
                       
        # Third plot: Onset and Return + Global length of signal
        plt.figure(figsize=(12, 8))  # Use the same figure size as the first plot
        # Plot the same data as before
        plt.plot(time, gfp, label="GFP (Original)", color='green', alpha=0.5)
        plt.plot(time, gfp_smooth_strong, label="GFP (Smoothed)", color='green')
        plt.plot(time, rfp, label="RFP (Original)", color='orange', alpha=0.5)
        plt.plot(time, rfp_smooth_strong, label="RFP (Smoothed)", color='orange')
        plt.plot(time, nir, label="NIR (Original)", color='darkred', alpha=0.5)
        plt.plot(time, nir_smooth_strong, label="NIR (Smoothed)", color='darkred')        
        # Mark detected onsets and return
        if gfp_onset_time: plt.axvline(gfp_onset_time, color='green', linestyle='--', label="GFP Onset")
        if rfp_onset_time: plt.axvline(rfp_onset_time, color='orange', linestyle='--', label="RFP Onset")
        if nir_onset_time: plt.axvline(nir_onset_time, color='darkred', linestyle='--', label="NIR Onset")        
        if gfp_return: plt.axvline(gfp_return, color='lime', linestyle='--', label="GFP Return Time")
        if rfp_return: plt.axvline(rfp_return, color='gold', linestyle='--', label="NIR Return Time")
        if nir_return: plt.axvline(nir_return, color='red', linestyle='--', label="RFP Return Time")                
        plt.xlabel("Time [sec]")
        plt.ylabel("Intensity")        
        plt.legend(loc='upper right', fontsize=6)        
        # Format the x-axis
        plt.xticks(np.arange(time[0], max(time), 30))
        # Add grid lines
        plt.grid(True, which='both', axis='x', color='lightgrey', linestyle='-', linewidth=0.2)
        plt.grid(True, which='both', axis='y', color='lightgrey', linestyle='-', linewidth=0.2)
        plt.yticks(fontsize=10)        
        plt.show()
    




Full batch analysis

In [ ]:

"""
CARMEN Trace Analysis Script – Onset, Return, and Oscillation Detection (Batch)
----------------------------------------------------------------------

This script performs batch analysis.

It is intended for batch processing after parameters have been optimised 
using a test/trial script. It processes every Excel file in a specified folder and analyses 
all 3-channel datasets within each file (e.g. columns C–E, F–H, ...), typically representing
individual ROIs or time series replicates.

Functionality:
- Reads `.xlsm` files from a user-defined folder.
- For each dataset (3 adjacent columns), performs:
  - Smoothing using Savitzky-Golay filter.
  - Onset detection using derivative peaks (via `scipy.signal.find_peaks`).
  - Return-to-baseline estimation with threshold- and duration-based logic.
  - Oscillation (peak/trough) detection between onset and return.
  - Duration calculation for individual oscillatory events based on derivative inflection points.
- Generates three summary plots per dataset and saves them as `.png`:
  - Zoomed view with onset markers
  - Oscillation overview with peak/trough markers
  - Global signal summary with onset and return lines
- Outputs:
  - Excel file with per-dataset results: onset times, return times, durations, oscillation counts
  - Text log file with all key parameter settings used in the analysis

Assumptions:
- Input files are `.xlsm` format with a sheet named 'Time Analysis'.
- Time is stored in column 'Protocol Time [sec]' (column B).
- Each dataset consists of three adjacent columns (GFP, RFP, NIR), starting from column C onward.
- Data may contain NaNs, which are filtered prior to processing.

Outputs:
- For each input file:
  - `<filename>_timing_results.xlsx`: Timing and oscillation results for each dataset
  - Three figures per dataset:
    - `<filename>_Data_Set_X_zoomed plot.png`
    - `<filename>_Data_Set_X_oscillations plot.png`
    - `<filename>_Data_Set_X_global plot.png`
  - `<filename>_analysis_log.txt`: Parameter documentation for reproducibility

Usage:
- Adjust parameters (e.g. smoothing window, baseline range, thresholds) in the parameter section.
- Set `folder_path` to the desired input directory.
- Run the script in a Python environment with the required packages.

"""

if not file_list:
    print(f"No '.xlsm' files found in the folder: {folder_path}")
else:
    print(f"Found {len(file_list)} files: {file_list}")

# Process each file one by one
for file_name in file_list:
    print(f"Processing file: {file_name}") 
    file_path = os.path.join(folder_path, file_name)
    # Load data from Excel
    df = pd.read_excel(file_path, sheet_name='Time Analysis')
    
    # Load time in seconds from column B
    time_seconds = df['Protocol Time [sec]'].to_numpy()
    
    data_sets = []  # To store data sets for the current file
    results = []  # To store the results for the current file

    # Load intensity data from columns C-D-E, F-G-H, etc.
    for i in range(0, len(df.columns) - 2, 3):  # Starting from column C
        intensity_set = df.iloc[:, 2 + i:5 + i].to_numpy()
        
        # Filter intensity data and time; Handle NaN values
        mask = ~np.isnan(intensity_set).any(axis=1)  # Mask to filter out rows with NaN values
        filtered_time = time_seconds[mask]           # Filter time values
        filtered_intensity = intensity_set[mask]     # Filter intensity values
        
        if len(filtered_intensity) > 0:  # Check if the intensity data is not empty
            data_sets.append((filtered_time, filtered_intensity))

    # Analyze each data set for the current file
    for idx, (time, intensity) in enumerate(data_sets):
        print(f"  Processing dataset {idx + 1} in file: {file_name}")
        gfp, rfp, nir = intensity.T  # Split intensity data into gfp, rfp, and nir
        
        # Step 1: Smooth the data using 
        gfp_smooth = savgol_filter(gfp, window_length, polyorder)
        rfp_smooth = savgol_filter(rfp, window_length, polyorder)
        nir_smooth = savgol_filter(nir, window_length, polyorder)
        # Stronger smooth the signal to reduce oscillations          
        gfp_smooth_strong = savgol_filter(gfp, window_length=window_length_strong, polyorder=polyorder_strong)
        rfp_smooth_strong = savgol_filter(rfp, window_length=window_length_strong, polyorder=polyorder_strong)
        nir_smooth_strong = savgol_filter(nir, window_length=window_length_strong, polyorder=polyorder_strong)
        
        # Apply a moving average filter after the Savitzky-Golay filter
        #def apply_moving_average(signal, window_size=10):
        #    smoothed_signal = pd.Series(signal).rolling(window=window_size, center=True).mean().to_numpy()
        #    return smoothed_signal
        #gfp_smooth_strong = apply_moving_average(gfp_smooth_strong, window_size=10)
        #rfp_smooth_strong = apply_moving_average(rfp_smooth_strong, window_size=10)
        #nir_smooth_strong = apply_moving_average(nir_smooth_strong, window_size=10)
        
        # Step 2: Compute the first derivative
        gfp_derivative = np.gradient(gfp_smooth, time)
        rfp_derivative = np.gradient(rfp_smooth, time)
        nir_derivative = np.gradient(nir_smooth, time)

        # Step 3: Detect onset times
        # Multiply the derivatives by the respective direction
        gfp_adjusted_derivative = gfp_peak_or_trough * gfp_derivative
        rfp_adjusted_derivative = rfp_peak_or_trough * rfp_derivative
        nir_adjusted_derivative = nir_peak_or_trough * nir_derivative
        #Detect peaks in the derivative
        gfp_peaks_derivative, _ = find_peaks(gfp_adjusted_derivative, **gfp_params)
        rfp_peaks_derivative, _ = find_peaks(rfp_adjusted_derivative, **rfp_params)
        nir_peaks_derivative, _ = find_peaks(nir_adjusted_derivative, **nir_params)
        # Assign onset times if peaks or troughs were detected        
        gfp_onset_time = time[gfp_peaks_derivative[0]] if len(gfp_peaks_derivative) > 0 else None
        rfp_onset_time = time[rfp_peaks_derivative[0]] if len(rfp_peaks_derivative) > 0 else None
        nir_onset_time = time[nir_peaks_derivative[0]] if len(nir_peaks_derivative) > 0 else None
        
        # Step 4: Compute return time and global durations
        # Function to define the baseline from a specific time range
        
        def define_baseline(signal, time, baseline_start_time, baseline_end_time):
            # Find the indices corresponding to the specified time range
            baseline_indices = np.where((time >= baseline_start_time) & (time <= baseline_end_time))[0]

            # Ensure that valid indices were found
            if len(baseline_indices) == 0:
                raise ValueError("No valid indices found for baseline calculation.")
            
            # Ensure the start time is before the end time
            if baseline_start_time >= baseline_end_time:
                raise ValueError("Baseline start time should be less than baseline end time.")
            
            # Check if indices are within the bounds of the signal
            if max(baseline_indices) >= len(signal):
                raise IndexError("Baseline indices are out of bounds of the signal length.")
            
            # Calculate and return the mean of the specified baseline range
            return np.mean(signal[baseline_indices])
        
        # Function to calculate signal duration from onset to baseline return
        def calculate_signal_duration(time, smoothed_signal, baseline, onset_time, signal_type, 
                                    is_low_baseline, min_consecutive_points=min_consecutive_points, 
                                    threshold_percentage=threshold_percentage):    
            # Handle the case where onset_time is None, e.g., no onset time could be identified automatically
            onset_time_candidate = onset_time
            if onset_time_candidate is None:
                print(f"{signal_type} onset_time is None. Using a fixed value of 60 seconds.")
                onset_time_candidate = 360  # Assign default onset_time

            # Find the onset index using a tolerance to match the onset time
            onset_index = (np.abs(time - onset_time_candidate)).argmin()

            # Start searching for the return to baseline at least 30 data points after the onset
            start_search_index = onset_index + 30

            # Define the lower and upper threshold based on the threshold percentage of the baseline
            lower_threshold = baseline * (1 - threshold_percentage)
            upper_threshold = baseline * (1 + threshold_percentage)

            # Initialize variables to track consecutive points within threshold
            consecutive_points = 0
            tracked_value_idx = None

            # Loop through the signal after the onset to find where the signal returns to baseline range
            for idx in range(start_search_index, len(smoothed_signal)):
                # Check if the signal is within the range of lower and upper thresholds
                if lower_threshold <= smoothed_signal[idx] <= upper_threshold:
                    consecutive_points += 1

                    # Track maximum or minimum value within the valid range based on `is_low_baseline`
                    if is_low_baseline:
                        # For GFP and NIR: Track minimum value
                        if tracked_value_idx is None or smoothed_signal[idx] < smoothed_signal[tracked_value_idx]:
                            tracked_value_idx = idx
                    else:
                        # For RFP: Track maximum value
                        if tracked_value_idx is None or smoothed_signal[idx] > smoothed_signal[tracked_value_idx]:
                            tracked_value_idx = idx

                else:
                    # Reset the consecutive points count if the signal leaves the threshold range
                    consecutive_points = 0
                    tracked_value_idx = None

                # If enough consecutive points are found, consider this a valid return to baseline
                if consecutive_points >= min_consecutive_points:
                    break  # Exit the loop once a valid return to baseline is found

            # Determine return time based on the tracked max or min value index
            if tracked_value_idx is not None:
                return_time = time[tracked_value_idx]
            else:
                return_time = np.nan  # If no valid return indices were found

            # Calculate the duration from onset to the return to baseline
            duration = return_time - onset_time_candidate

            # Return the onset time, return time, and calculated duration
            return onset_time_candidate, return_time, duration

        # Calculate baselines
        
        gfp_baseline = define_baseline(gfp, time, baseline_start_time, baseline_end_time)
        print(f"GFP baseline calculated for the range {baseline_start_time}s to {baseline_end_time}s: {gfp_baseline}")

        nir_baseline = define_baseline(nir, time, baseline_start_time, baseline_end_time)
        print(f"NIR baseline calculated for the range {baseline_start_time}s to {baseline_end_time}s: {nir_baseline}")

        rfp_baseline = define_baseline(rfp, time, baseline_start_time, baseline_end_time)
        print(f"RFP baseline calculated for the range {baseline_start_time}s to {baseline_end_time}s: {rfp_baseline}")
      
        # Determine if the baseline is low or high for each signal
        low_baseline_threshold = 20
        is_low_baseline_gfp = gfp_baseline < low_baseline_threshold
        is_low_baseline_nir = nir_baseline < low_baseline_threshold
        is_low_baseline_rfp = rfp_baseline < low_baseline_threshold

        # Calculate durations for GFP, RFP, and NIR
        # GFP: Determine if it's a low baseline and calculate return time accordingly
        gfp_onset_time, gfp_return, gfp_globalduration = calculate_signal_duration(
            time, gfp_smooth_strong, gfp_baseline, gfp_onset_time, signal_type='GFP', is_low_baseline=is_low_baseline_gfp)

        # NIR: Determine if it's a low baseline and calculate return time accordingly
        nir_onset_time, nir_return, nir_globalduration = calculate_signal_duration(
            time, nir_smooth_strong, nir_baseline, nir_onset_time, signal_type='NIR', is_low_baseline=is_low_baseline_nir)

        # RFP: Determine if it's a low baseline and calculate return time accordingly
        rfp_onset_time, rfp_return, rfp_globalduration = calculate_signal_duration(
            time, rfp_smooth_strong, rfp_baseline, rfp_onset_time, signal_type='RFP', is_low_baseline=is_low_baseline_rfp)

        # Print results
        print(f"GFP: Onset={gfp_onset_time}, Return={gfp_return}, Duration={gfp_globalduration}")
        print(f"NIR: Onset={nir_onset_time}, Return={nir_return}, Duration={nir_globalduration}")
        print(f"RFP: Onset={rfp_onset_time}, Return={rfp_return}, Duration={rfp_globalduration}")

        if gfp_onset_time and rfp_onset_time and nir_onset_time:
            delay_nir_rfp = rfp_onset_time - nir_onset_time
            delay_gfp_rfp = gfp_onset_time - rfp_onset_time
            delay_nir_gfp = gfp_onset_time - nir_onset_time   
        else:
            delay_nir_rfp = delay_gfp_rfp = delay_nir_gfp = None

        print(f"Delay NIR to RFP {delay_nir_rfp}, Delay GFP to RFP {delay_gfp_rfp}, Delay NIR to GFP {delay_nir_gfp}")
                
        # Step 5: Oscillation Detection and Count (Peaks and Troughs)
        # Function to detect peaks and troughs for a given signal

        def detect_oscillations(signal, prominence, distance, onset_time=None, return_time=None, lower_threshold=None, upper_threshold=None):
            """
            Detect peaks and troughs in the given signal using specified prominence, distance parameters,
            and optional lower and upper thresholds to reduce noise-induced detections near baseline.
            Additionally, only consider oscillations between onset and return times.

            Parameters:
            - signal: The input signal (numpy array)
            - prominence: The prominence of the peaks/troughs for detection
            - distance: Minimum distance between peaks/troughs
            - onset_time: The time to start looking for oscillations (e.g., after the onset)
            - return_time: The time to stop looking for oscillations (e.g., before the return time)
            - lower_threshold: Lower threshold value to ignore peaks/troughs near the baseline
            - upper_threshold: Upper threshold value to ignore peaks/troughs near the baseline

            Returns:
            - peaks: Indices of the peaks in the signal (between onset and return time)
            - troughs: Indices of the troughs in the signal (between onset and return time)
            """
            peaks = find_peaks(signal, prominence=prominence, distance=distance)[0]
            troughs = find_peaks(-signal, prominence=prominence, distance=distance)[0]

            # Convert onset and return times to indices
            if onset_time is not None:
                onset_index = (np.abs(time - onset_time)).argmin()
            else:
                onset_index = 0

            if return_time is not None:
                return_index = (np.abs(time - return_time)).argmin()
            else:
                return_index = len(signal)

            # Filter peaks and troughs to only include those between onset and return times
            peaks = [p for p in peaks if onset_index <= p <= return_index]
            troughs = [t for t in troughs if onset_index <= t <= return_index]

            # Filter out peaks and troughs that are within the lower and upper thresholds if provided
            #if lower_threshold is not None and upper_threshold is not None:
            #    peaks = [p for p in peaks if signal[p] < lower_threshold or signal[p] > upper_threshold]
            #    troughs = [t for t in troughs if signal[t] < lower_threshold or signal[t] > upper_threshold]

            if len(peaks) == 0 or len(troughs) == 0:
                print("Warning: No valid peaks or troughs detected for the signal.")

            return peaks, troughs

        # Detection and Duration Calculation for Each Signal (with onset and return time check)

        # Detect peaks and troughs for GFP signal (only between onset and return time)
        gfp_peaks, gfp_troughs = detect_oscillations(
            gfp_smooth_strong, prominence=gfp_peak_prominence, distance=gfp_peak_distance, onset_time=gfp_onset_time, return_time=gfp_return
        )
        gfp_oscillations = min(len(gfp_peaks), len(gfp_troughs))  # Each oscillation requires one peak and one trough

        # Detect peaks and troughs for RFP signal (only between onset and return time)
        rfp_peaks, rfp_troughs = detect_oscillations(
            rfp_smooth_strong, prominence=rfp_peak_prominence, distance=rfp_peak_distance, onset_time=rfp_onset_time, return_time=rfp_return
        )
        rfp_oscillations = min(len(rfp_peaks), len(rfp_troughs))

        # Detect peaks and troughs for NIR signal (only between onset and return time)
        nir_peaks, nir_troughs = detect_oscillations(
            nir_smooth_strong, prominence=nir_peak_prominence, distance=nir_peak_distance, onset_time=nir_onset_time, return_time=nir_return
        )
        nir_oscillations = min(len(nir_peaks), len(nir_troughs))

        # Function to calculate oscillation durations from the derivative of the signal
        def calculate_durations(derivative, time, signal_name, onset_time=None, return_time=None):
            """
            Calculate the durations of oscillations based on the changes in the derivative of the signal,
            considering only the oscillations between onset and return times.

            Parameters:
            - derivative: The derivative of the smoothed signal (numpy array)
            - time: Time vector corresponding to the signal (numpy array)
            - signal_name: Name of the signal being processed (str)
            - onset_time: The time to start calculating oscillations (e.g., after the onset)
            - return_time: The time to stop calculating oscillations (e.g., before the return time)

            Returns:
            - signal_durations: List of calculated durations for each oscillation
            """
            # Determine regions where the derivative changes sign (+1 for increase, -1 for decrease)
            change_regions = np.sign(derivative)

            # Detect changes in regions using np.diff; prepend zero to maintain the length
            transitions = np.diff(change_regions, prepend=0)

            # Identify indices where transitions occur, indicating peaks or troughs
            starts = np.where(transitions != 0)[0]

            # Convert onset and return times to indices
            if onset_time is not None:
                onset_index = (np.abs(time - onset_time)).argmin()
            else:
                onset_index = 0

            if return_time is not None:
                return_index = (np.abs(time - return_time)).argmin()
            else:
                return_index = len(derivative)

            # Filter out transitions that are outside the onset to return range
            starts = starts[(starts >= onset_index) & (starts <= return_index)]

            # Initialize list to store durations of detected oscillations
            signal_durations = []

            # Loop through the transitions to calculate each oscillation duration
            for i in range(len(starts) - 1):
                start_idx, end_idx = starts[i], starts[i + 1]

                # Validate that indices are within bounds of the time array
                if end_idx < len(time):
                    duration = int(time[end_idx] - time[start_idx])  # Convert duration to Python int
                    signal_durations.append(duration)
                else:
                    print(f"Warning: Index {end_idx} is out of bounds for time array of length {len(time)}.")

            return signal_durations

        # Compute durations for each signal (considering onset to return time)
        durations = {}

        # Calculate durations for each signal derivative (GFP, RFP, NIR)
        for derivative, name, onset_time, return_time in zip(
            [gfp_derivative, rfp_derivative, nir_derivative], 
            ["GFP", "RFP", "NIR"], 
            [gfp_onset_time, rfp_onset_time, nir_onset_time],
            [gfp_return, rfp_return, nir_return]
        ):
            durations[name] = calculate_durations(derivative, time, name, onset_time=onset_time, return_time=return_time)
        
        
        # Step 6: Append results
        results.append([
            idx + 1,
            gfp_onset_time, rfp_onset_time, nir_onset_time,
            gfp_return, rfp_return, nir_return, 
            gfp_oscillations, rfp_oscillations, nir_oscillations,
            durations.get("GFP", []), durations.get("RFP", []), durations.get("NIR", []), 
            gfp_globalduration, rfp_globalduration, nir_globalduration
        ])
        # could also directly append: delay_nir_rfp, delay_gfp_rfp, delay_nir_gfp,
        # Step 7: Plot curves, smoothed curves and derivative curves
        # First plot: Zoomed in on the time range from 120 to 180
        plt.figure(figsize=(12, 8))  # Use the same figure size as the first plot
        plt.plot(time, gfp, label="GFP (Original)", color='green', alpha=0.5)
        plt.plot(time, gfp_smooth, label="GFP (Smoothed)", color='green')
        plt.plot(time, gfp_derivative, label="GFP Derivative", color='lime', linestyle='--')        
        plt.plot(time, rfp, label="RFP (Original)", color='orange', alpha=0.5)
        plt.plot(time, rfp_smooth, label="RFP (Smoothed)", color='orange')
        plt.plot(time, rfp_derivative, label="RFP Derivative", color='gold', linestyle='--')
        plt.plot(time, nir, label="NIR (Original)", color='darkred', alpha=0.5)
        plt.plot(time, nir_smooth, label="NIR (Smoothed)", color='darkred')
        plt.plot(time, nir_derivative, label="NIR Derivative", color='red', linestyle='--')   
        # Mark detected onsets
        if gfp_onset_time: plt.axvline(gfp_onset_time, color='green', linestyle='--', label="GFP Onset")
        if rfp_onset_time: plt.axvline(rfp_onset_time, color='orange', linestyle='--', label="RFP Onset")
        if nir_onset_time: plt.axvline(nir_onset_time, color='darkred', linestyle='--', label="NIR Onset")        
        plt.xlabel("Time [sec]")
        plt.ylabel("Intensity")        
        plt.legend(loc='upper left', fontsize=6)        
        
        #Set the x-axis limits to focus on the specified range in seconds
        plt.xlim(zoomed_min, zoomed_max)
        
        # Set the grid for x-axis with major and minor ticks
        plt.grid(True, which='major', axis='x', color='lightgrey', linestyle='-', linewidth=0.5)  # Major grid lines
        plt.grid(True, which='minor', axis='x', color='lightgrey', linestyle='--', linewidth=0.3)  # Minor grid lines

        # Set major ticks (labels) every 10 seconds
        plt.gca().xaxis.set_major_locator(MultipleLocator(10))

        # Set minor ticks (grid lines) every 2 seconds
        plt.gca().xaxis.set_minor_locator(MultipleLocator(2))

        # Format minor ticks on the x-axis
        plt.tick_params(axis='x', which='minor', length=4, width=0.8, labelsize=8, labelbottom=False)  # Minor ticks, no labels
        plt.tick_params(axis='x', which='major', length=6, width=1.2, labelsize=10)  # Major ticks with labels

        # Set grid for y-axis
        plt.grid(True, which='both', axis='y', color='lightgrey', linestyle='-', linewidth=0.5)


        # Save the plot as PNG file
        plot1_filename = os.path.join(folder_path, f"{os.path.splitext(file_name)[0]}_Data_Set_{idx + 1}_zoomed plot.png")
        plt.title(f"Signal Analysis - {os.path.splitext(file_name)[0]}_Data_Set_{idx + 1}")
        plt.savefig(plot1_filename)
        plt.close()
        #plt.show()
                
        #Second Plot: Oscillations        
        plt.figure(figsize=(12, 8))        
        plt.plot(time, gfp, label="GFP (Original)", color='green', alpha=0.5)
        plt.plot(time, gfp_smooth_strong, label="GFP (Smoothed)", color='green')
        plt.plot(time, gfp_derivative, label="GFP Derivative", color='lime', linestyle='--')        
        plt.plot(time, rfp, label="RFP (Original)", color='orange', alpha=0.5)
        plt.plot(time, rfp_smooth_strong, label="RFP (Smoothed)", color='orange')
        plt.plot(time, rfp_derivative, label="RFP Derivative", color='gold', linestyle='--')        
        plt.plot(time, nir, label="NIR (Original)", color='darkred', alpha=0.5)
        plt.plot(time, nir_smooth_strong, label="NIR (Smoothed)", color='darkred')
        plt.plot(time, nir_derivative, label="NIR Derivative", color='red', linestyle='--')
        # Mark detected onsets and return
        #if gfp_onset_time: plt.axvline(gfp_onset_time, color='green', linestyle='--', label="GFP Onset")
        #if rfp_onset_time: plt.axvline(rfp_onset_time, color='orange', linestyle='--', label="RFP Onset")
        #if nir_onset_time: plt.axvline(nir_onset_time, color='darkred', linestyle='--', label="NIR Onset")
        #if gfp_return: plt.axvline(gfp_return, color='lime', linestyle='--', label="GFP Return Time")
        #if rfp_return: plt.axvline(rfp_return, color='gold', linestyle='--', label="RFP Return Time")
        #if nir_return: plt.axvline(nir_return, color='red', linestyle='--', label="NIR Return Time")
        #Mark detected peaks with scatter points
        for peak in gfp_peaks:
            plt.scatter(time[peak], gfp[peak], color='green', zorder=5, label="GFP Peak" if peak == gfp_peaks[0] else "")  # Only label the first peak
        for peak in rfp_peaks:
            plt.scatter(time[peak], rfp[peak], color='orange', zorder=5, label="RFP Peak" if peak == rfp_peaks[0] else "")  # Only label the first peak
        for peak in nir_peaks:
            plt.scatter(time[peak], nir[peak], color='darkred', zorder=5, label="NIR Peak" if peak == nir_peaks[0] else "")  # Only label the first peak
        #Mark detected troughs with scatter points
        for trough in gfp_troughs:
            plt.scatter(time[trough], gfp[trough], color='lime', zorder=5, label="GFP Trough" if trough == gfp_troughs[0] else "")  # Only label the first trough
        for trough in rfp_troughs:
            plt.scatter(time[trough], rfp[trough], color='gold', zorder=5, label="RFP Trough" if trough == rfp_troughs[0] else "")  # Only label the first trough
        for trough in nir_troughs:
            plt.scatter(time[trough], nir[trough], color='red', zorder=5, label="NIR Trough" if trough == nir_troughs[0] else "")  # Only label the first trough
        # Boolean flags to control labeling
        gfp_peak_label = True
        rfp_peak_label = True
        nir_peak_label = True
        gfp_trough_label = True
        rfp_trough_label = True
        nir_trough_label = True
        #label and legend
        plt.xlabel("Time [sec]", fontsize=10)
        plt.ylabel("Intensity", fontsize=10)
        plt.legend(loc='upper right', fontsize=6) 
        # Format the x-axis: label every 5 seconds starting from the first data point
        plt.xticks(np.arange(time[0], max(time), 30))  # Set x-ticks every 30 seconds starting from the first time point
        plt.grid(True, which='both', axis='x', color='lightgrey', linestyle='-', linewidth=0.2)
        plt.grid(True, which='both', axis='y', color='lightgrey', linestyle='-', linewidth=0.2)
        plt.yticks(fontsize=10)
        # Save the plot as PNG file
        plot2_filename = os.path.join(folder_path, f"{os.path.splitext(file_name)[0]}_Data_Set_{idx + 1}_oscillations plot.png")
        plt.title(f"Signal Analysis - {os.path.splitext(file_name)[0]}_Data_Set_{idx + 1}")
        plt.savefig(plot2_filename)
        plt.close()
        #plt.show()
                       
        # Third plot: Onset and Return + Global length of signal
        plt.figure(figsize=(12, 8))  # Use the same figure size as the first plot
        # Plot the same data as before        
        plt.plot(time, gfp, label="GFP (Original)", color='green', alpha=0.5)
        plt.plot(time, gfp_smooth_strong, label="GFP (Smoothed)", color='green')
        plt.plot(time, gfp_derivative, label="GFP Derivative", color='lime', linestyle='--')        
        plt.plot(time, rfp, label="RFP (Original)", color='orange', alpha=0.5)
        plt.plot(time, rfp_smooth_strong, label="RFP (Smoothed)", color='orange')
        plt.plot(time, rfp_derivative, label="RFP Derivative", color='gold', linestyle='--')        
        plt.plot(time, nir, label="NIR (Original)", color='darkred', alpha=0.5)
        plt.plot(time, nir_smooth_strong, label="NIR (Smoothed)", color='darkred')
        plt.plot(time, nir_derivative, label="NIR Derivative", color='red', linestyle='--')
        
        # Mark detected onsets and return
        if gfp_onset_time: plt.axvline(gfp_onset_time, color='green', linestyle='--', label="GFP Onset")
        if rfp_onset_time: plt.axvline(rfp_onset_time, color='orange', linestyle='--', label="RFP Onset")
        if nir_onset_time: plt.axvline(nir_onset_time, color='darkred', linestyle='--', label="NIR Onset")        
        if gfp_return: plt.axvline(gfp_return, color='lime', linestyle='--', label="GFP Return Time")
        if rfp_return: plt.axvline(rfp_return, color='gold', linestyle='--', label="NIR Return Time")
        if nir_return: plt.axvline(nir_return, color='red', linestyle='--', label="RFP Return Time")                
        plt.xlabel("Time [sec]")
        plt.ylabel("Intensity")        
        plt.legend(loc='upper right', fontsize=6)        
        # Format the x-axis
        plt.xticks(np.arange(time[0], max(time), 30))
        # Add grid lines
        plt.grid(True, which='both', axis='x', color='lightgrey', linestyle='-', linewidth=0.2)
        plt.grid(True, which='both', axis='y', color='lightgrey', linestyle='-', linewidth=0.2)
        plt.yticks(fontsize=10)        
        # Save the plot as PNG file
        plot3_filename = os.path.join(folder_path, f"{os.path.splitext(file_name)[0]}_Data_Set_{idx + 1}_global plot.png")
        plt.title(f"Signal Analysis - {os.path.splitext(file_name)[0]}_Data_Set_{idx + 1}")
        plt.savefig(plot3_filename)
        plt.close()
        #plt.show()
      
    # Save results to Excel (update headers to include new metrics)
    output_file_path = os.path.join(folder_path, f"{os.path.splitext(file_name)[0]}_timing_results.xlsx")
    results_df = pd.DataFrame(results, columns=[
        "Data Set", "GFP Onset Time", "RFP Onset Time", "NIR Onset Time",
        "GFP Return Time", "RFP Return Time", "NIR Return Time",
        "GFP Oscillations", "RFP Oscillations", "NIR Oscillations",
        "GFP Peak Durations", "RFP Peak Durations", "NIR Peak Durations",
        "GFP Global Duration", "RFP Global Duration", "NIR Global Duration"
    ])
    #"Delay RFP-NIR", "Delay GFP-RFP", "Delay GFP-NIR",
    results_df.to_excel(output_file_path, index=False)
    print(f"Results saved to {output_file_path}")

# Define the log file path
log_file_path = os.path.join(folder_path, f"{os.path.splitext(file_name)[0]}_analysis_log.txt")

# Function to log parameters to a file
def log_parameters_to_file(file_path):
    # Open the file in write mode to overwrite its content
    with open(file_path, 'w') as file:
        # Write constants for smoothing with Savitzky-Golay filter
        file.write(f"window_length = {window_length}\n")
        file.write(f"polyorder = {polyorder}\n")
        file.write(f"window_length_strong = {window_length_strong}\n")
        file.write(f"polyorder_strong = {polyorder_strong}\n")

        # Write threshold and min consecutive points for return calculation
        file.write(f"threshold_percentage = {threshold_percentage}\n")
        file.write(f"min_consecutive_points = {min_consecutive_points}\n")

        # Write baseline time range
        file.write(f"baseline_start_time = {baseline_start_time}\n")
        file.write(f"baseline_end_time = {baseline_end_time}\n")

        # Write parameters for onset peak detection
        file.write(f"gfp_params = {gfp_params}\n")
        file.write(f"rfp_params = {rfp_params}\n")
        file.write(f"nir_params = {nir_params}\n")

        # Write oscillation detection parameters
        file.write(f"gfp_peak_prominence = {gfp_peak_prominence}\n")
        file.write(f"gfp_peak_distance = {gfp_peak_distance}\n")
        file.write(f"rfp_peak_prominence = {rfp_peak_prominence}\n")
        file.write(f"rfp_peak_distance = {rfp_peak_distance}\n")
        file.write(f"nir_peak_prominence = {nir_peak_prominence}\n")
        file.write(f"nir_peak_distance = {nir_peak_distance}\n")

# Call the function to log parameters
log_parameters_to_file(log_file_path)
print(f"Variables and constants logged to {log_file_path}")
